In [1]:
import socket
import struct

# API

In [2]:
INFO = b"UR001!"
ID = b"UL!"
FIRMWARE = b"UN!"
POWER_ON = b"UK001!"
POWER_OFF = b"UK004!"

def set_grill_temp(temp_f):
    if not (150 <= temp_f <= 450):
        raise ValueError("invalid temperature")
    return b"UT???!".replace(b"???", format(temp_f, "03d").encode("ascii"))

def set_probe_temp(temp_f):
    if not (50 <= temp_f <= 220):
        raise ValueError("invalid temperature")
    return b"UF{}!"

def broadcast():
    return "UH{}{}{}{}{}!".format(0, len(ssid), ssid, len(password), password)

In [3]:
set_grill_temp(234)

b'UT234!'

In [4]:
import base64
import fcntl
import ipaddress
import socket
import subprocess

In [5]:
socket.if_nameindex()

[(1, 'lo'),
 (2, 'wwan0'),
 (3, 'wlp58s0'),
 (4, 'tailscale0'),
 (5, 'anbox0'),
 (6, 'br-8df54316e2a9'),
 (7, 'br-63aed09d3259'),
 (8, 'br-73fb78ea0798'),
 (9, 'docker0'),
 (10, 'ens1')]

In [6]:
iface = "wlp58s0"
socket.inet_ntoa(
    fcntl.ioctl(
        socket.socket(socket.AF_INET, socket.SOCK_DGRAM),
        35099,
        struct.pack('256s', iface.encode("ascii"))
    )[20:24]
)

'255.255.255.0'

In [7]:
class MAC:
    """MAC or MAC Prefix"""
    def __init__(self, address):
        self.raw = base64.b16decode(address.replace(":", ""), casefold=True)

    def __repr__(self):
        return f"{self.__class__.__name__}(address='{self}')"

    def __str__(self):
        return ":".join(hex(b)[2:] for b in self.raw)
    
    def __contains__(self, other):
        """Does this MAC prefix contain the provided MAC?"""
        return other.raw.startswith(self.raw)

In [8]:
x = MAC("7C:a7")
x in MAC("7c")

True

In [9]:
base64.b16decode("7C:a7:b0".replace(":", ""), casefold=True)

b'|\xa7\xb0'

In [10]:
def local_macs(cidr):
    network = ipaddress.IPv4Network(cidr)
    # attempt connection to everything to populate ARP
    subprocess.check_output(["nmap", "-sP", str(network)])
    
    # dump ARP table
    neighbors = subprocess.check_output(["ip", "neighbor", "show"], universal_newlines=True)
    
    # and parse output from `ip neigh`
    arp_pairs = []
    for line in neighbors.splitlines():
        ip, _dev, interface, result = line.split(" ", 3)
        result = result.strip()
        if "lladdr" not in result:
            # FAILED, INCOMPLETE, etc.
            continue
        # REACHABLE, STALE, DELAY, etc.
        _lladdr, mac, status = result.split()
        arp_pairs.append((ip, mac))

    return arp_pairs

In [11]:
macs = local_macs("192.168.42.0/24")

In [12]:
gmg_mac = "7c:a7:b0:af:4e:58"
mac_to_ip = {mac: ip for ip, mac in macs}

mac_to_ip[gmg_mac]

'192.168.42.245'

In [13]:
def send_command(command: bytes, ip, port=8080):
    address = (ip, port)
    with socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM) as sock:
        sock.connect(address)
        sock.send(command)
        return sock.recv(128)

In [14]:
send_command(ID, mac_to_ip[gmg_mac])[:5]

b'GMG12'

In [15]:
send_command(FIRMWARE, mac_to_ip[gmg_mac])[:5]

b'UNDB0'

In [16]:
send_command(INFO, mac_to_ip[gmg_mac])

b'UR\x12\x00Y\x02\x96\x00\x05\n\x142\x19\x19\x19\x19\x17\x00\x00\x00\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x03'

In [17]:
import base64
import struct

In [19]:
info_data = send_command(INFO, mac_to_ip[gmg_mac])
len(info_data)

36

In [20]:
info_data

b'UR\x12\x00Y\x02\x96\x00\x05\n\x142\x19\x19\x19\x19\x16\x00\x00\x00\xff\xff\xff\xff\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x03'

In [21]:
base64.b16encode(info_data)

b'5552120059029600050A14321919191916000000FFFFFFFF000000000000000001000003'

In [24]:
import subprocess

def xxd(buf):
    """call xxd on some binary"""
    p = subprocess.Popen(['xxd', "-"], stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout_data = p.communicate(input=buf)[0].decode()
    print(stdout_data)

In [25]:
xxd(info_data)

00000000: 5552 1200 5902 9600 050a 1432 1919 1919  UR..Y......2....
00000010: 1600 0000 ffff ffff 0000 0000 0000 0000  ................
00000020: 0100 0003                                ....



# Grill Data Format

In [75]:
spec = [
    ("2x", "?"), # always "UR"
    ("h", "grill_temp"),   # 2-3
    ("h", "probe1_temp"),  # 4-5
    ("h", "grill_target"), # 6-7
    ("b", "_08"), # 8
    ("b", "_09"), # 9
    ("b", "_10"), # 10
    ("b", "_11"), # 11
    ("b", "_12"), # 12
    ("b", "_13"), # 13
    ("b", "_14"), # 14
    ("b", "_15"), # 15
    ("h", "probe2_temp"),
    ("h", "probe2_target"),
    ("b", "curve_remain_time"),
    ("b", "_21"),
    ("b", "_22"),
    ("b", "_23"),
    ("b", "warn_code"),
    ("b", "_25"),
    ("b", "_26"),
    ("b", "_27"),
    ("h", "probe1_target"),
    ("b", "grill_state"), # 30
    ("b", "grill_mode"),
    ("b", "fire_state"),
    ("b", "fire_state_pct"),
    ("b", "_34"),
    ("b", "_35"),
]

In [27]:
def specunpack(spec, buf, byte_order=""):
    """
    Given a 'spec' for a structure which is a sequence of tuple pairs, each
    consisting of a struct format character and a 'name'. On unpacking 'buf',
    the names are used in the returned dictionary.

    Pad characters ('x') are ignored; the name in those pairs is irrelevant.
    """
    format_ = [byte_order]
    names = set()
    for part, name in spec:
        format_.append(part)
        if "x" in part:
            continue
        if name in names:
            raise ValueError(f"duplicate names: {name}")
        names.add(name)
    unpacked = struct.unpack("".join(format_), buf)
    output = {}
    for name, data in zip((name for part, name in spec if "x" not in part), unpacked):
        output[name] = data
    return output

In [66]:
specunpack(spec, info_data, byte_order="<")

{'grill_temp': 18,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 22,
 'probe2_target': 0,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 0,
 'grill_state': 0,
 'grill_mode': 0,
 'fire_state': 1,
 'fire_state_pct': 0}

# Reverse Engineering Fields

In [30]:
import datetime

Nothing plugged in to probes, grill off

In [216]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
docunpack(spec, data)

2022-02-09T00:22:43.839540

00000000: 5552 2500 5902 9600 050a 1432 1919 1919  UR%.Y......2....
00000010: 5902 0000 ffff ffff 0000 0000 0000 0000  Y...............
00000020: 0100 0003                                ....



{'grill_temp': 37,
 'food_temp': 601,
 'grill_target': 150,
 'curve_remain_time': -1,
 'warn_code': 0,
 'food_target': 0,
 'grill_state': 0,
 'grill_mode': 0,
 'fire_state': 1,
 'fire_state_pct': 0}

Probe 1 plugged in

In [218]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
docunpack(spec, data)

2022-02-09T00:24:16.204785

00000000: 5552 2500 3d00 9600 050a 1432 1919 1919  UR%.=......2....
00000010: 5902 0000 ffff ffff 0000 0000 0000 0000  Y...............
00000020: 0100 0003                                ....



{'grill_temp': 37,
 'food_temp': 61,
 'grill_target': 150,
 'curve_remain_time': -1,
 'warn_code': 0,
 'food_target': 0,
 'grill_state': 0,
 'grill_mode': 0,
 'fire_state': 1,
 'fire_state_pct': 0}

Probe 1 unplugged, probe 2 plugged in

In [225]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
docunpack(spec, data)

2022-02-09T00:27:52.654772

00000000: 5552 2500 5902 9600 050a 1432 1919 1919  UR%.Y......2....
00000010: 2400 0000 ffff ffff 0000 0000 0000 0000  $...............
00000020: 0100 0003                                ....



{'grill_temp': 37,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 36,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 0,
 'grill_state': 0,
 'grill_mode': 0,
 'fire_state': 1,
 'fire_state_pct': 0}

Another day... grill off, probe plugged in to #2

In [32]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:36:07.401382

00000000: 5552 1200 5902 9600 050a 1432 1919 1919  UR..Y......2....
00000010: 1600 0000 ffff ffff 0000 0000 0000 0000  ................
00000020: 0100 0003                                ....



{'grill_temp': 18,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 22,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 0,
 'grill_state': 0,
 'grill_mode': 0,
 'fire_state': 1,
 'fire_state_pct': 0}

Connecting app to grill

In [33]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:36:50.731017

00000000: 5552 1200 5902 9600 050a 1432 1919 1919  UR..Y......2....
00000010: 1600 0000 ffff ffff 0000 0000 0000 0000  ................
00000020: 0100 0003                                ....



{'grill_temp': 18,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 22,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 0,
 'grill_state': 0,
 'grill_mode': 0,
 'fire_state': 1,
 'fire_state_pct': 0}

Grill on (shows "1" on display)

In [34]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:37:14.304276

00000000: 5552 1300 5902 9600 050a 1432 1919 1919  UR..Y......2....
00000010: 1600 0000 ffff ffff 0000 0000 0000 0100  ................
00000020: 0219 0003                                ....



{'grill_temp': 19,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 22,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 0,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 25}

In [35]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:37:26.009419

00000000: 5552 1300 5902 9600 050a 1432 1919 1919  UR..Y......2....
00000010: 1500 0000 ffff ffff 0000 0000 0000 0100  ................
00000020: 0219 0003                                ....



{'grill_temp': 19,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 21,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 0,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 25}

In [46]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:38:44.247647

00000000: 5552 1300 5902 9600 050a 1432 1919 1919  UR..Y......2....
00000010: 1500 0000 ffff ffff 0000 0000 0000 0100  ................
00000020: 0219 0003                                ....



{'grill_temp': 19,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 21,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 0,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 25}

After showing "2" on display

In [49]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:39:50.258786

00000000: 5552 1300 5902 9600 050a 1432 1919 1919  UR..Y......2....
00000010: 1500 0000 ffff ffff 0000 0000 0000 0100  ................
00000020: 0219 0003                                ....



{'grill_temp': 19,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 21,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 0,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 25}

Setting food temp 1 to 111, food temp 2 to 222

In [64]:
hex(111), hex(222)

('0x6f', '0xde')

In [51]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:40:32.447959

00000000: 5552 1500 5902 9600 050a 1432 1919 1919  UR..Y......2....
00000010: 1700 de00 ffff ffff 0000 0000 6f00 0100  ............o...
00000020: 0219 0003                                ....



{'grill_temp': 21,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 23,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 25}

Turning on Pizza mode

In [53]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:41:56.134064

00000000: 5552 2200 5902 9600 052a 1432 1919 1919  UR".Y....*.2....
00000010: 2a00 de00 ffff ffff 0000 0000 6f00 0100  *...........o...
00000020: 0232 0003                                .2..



{'grill_temp': 34,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 42,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 50}

Pizza mode off (grill also says it's on now, reporting temp on front panel)

In [54]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:42:09.115424

00000000: 5552 2300 5902 9600 050a 1432 1919 1919  UR#.Y......2....
00000010: 2f00 de00 ffff ffff 0000 0000 6f00 0100  /...........o...
00000020: 0232 0003                                .2..



{'grill_temp': 35,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 47,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 50}

Climate setting = "Cold" (was average)

In [55]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:43:33.576489

00000000: 5552 3400 5902 9600 0506 1432 1919 1919  UR4.Y......2....
00000010: 4600 de00 ffff ffff 0000 0000 6f00 0100  F...........o...
00000020: 0232 0003                                .2..



{'grill_temp': 52,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 70,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 50}

Climate setting = "Cold" (was average)

In [55]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:43:33.576489

00000000: 5552 3400 5902 9600 0506 1432 1919 1919  UR4.Y......2....
00000010: 4600 de00 ffff ffff 0000 0000 6f00 0100  F...........o...
00000020: 0232 0003                                .2..



{'grill_temp': 52,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 70,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 50}

Climate setting = "Icy" (was "cold")

In [56]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:43:58.784060

00000000: 5552 3a00 5902 9600 0502 1432 1919 1919  UR:.Y......2....
00000010: 5100 de00 ffff ffff 0000 0000 6f00 0100  Q...........o...
00000020: 0232 0003                                .2..



{'grill_temp': 58,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 81,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 50}

Climate setting = "Icy" (was "cold")

In [56]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:43:58.784060

00000000: 5552 3a00 5902 9600 0502 1432 1919 1919  UR:.Y......2....
00000010: 5100 de00 ffff ffff 0000 0000 6f00 0100  Q...........o...
00000020: 0232 0003                                .2..



{'grill_temp': 58,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 81,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 50}

Climate setting = "warm" (was "icy")

In [57]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:44:20.314608

00000000: 5552 3f00 5902 9600 050e 1432 1919 1919  UR?.Y......2....
00000010: 5a00 de00 ffff ffff 0000 0000 6f00 0100  Z...........o...
00000020: 0232 0003                                .2..



{'grill_temp': 63,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 90,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 50}

Climate setting = "warm" (was "icy")

In [57]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:44:20.314608

00000000: 5552 3f00 5902 9600 050e 1432 1919 1919  UR?.Y......2....
00000010: 5a00 de00 ffff ffff 0000 0000 6f00 0100  Z...........o...
00000020: 0232 0003                                .2..



{'grill_temp': 63,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 90,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 50}

Climate setting = "hot" (was "warm")

In [58]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:44:34.854549

00000000: 5552 4200 5902 9600 0512 1432 1919 1919  URB.Y......2....
00000010: 5f00 de00 ffff ffff 0000 0000 6f00 0100  _...........o...
00000020: 0232 0003                                .2..



{'grill_temp': 66,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 95,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 50}

Climate setting = "hot" (was "warm")

In [58]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:44:34.854549

00000000: 5552 4200 5902 9600 0512 1432 1919 1919  URB.Y......2....
00000010: 5f00 de00 ffff ffff 0000 0000 6f00 0100  _...........o...
00000020: 0232 0003                                .2..



{'grill_temp': 66,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 95,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 50}

Climate setting = "average" (was "hot")

In [59]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:44:48.612005

00000000: 5552 4600 5902 9600 050a 1432 1919 1919  URF.Y......2....
00000010: 6600 de00 ffff ffff 0000 0000 6f00 0100  f...........o...
00000020: 0232 0003                                .2..



{'grill_temp': 70,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 102,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 50}

Grill target temp = 225 (was 150)

In [61]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:45:33.589294

00000000: 5552 5500 5902 e100 050a 1432 1919 1919  URU.Y......2....
00000010: 8000 de00 ffff ffff 0000 0000 6f00 0100  ............o...
00000020: 0232 0003                                .2..



{'grill_temp': 85,
 'probe1_temp': 601,
 'grill_target': 225,
 'probe2_temp': 128,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 50}

Grill off ("FAN")

In [62]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:47:22.757157

00000000: 5552 7400 5902 e100 050a 1432 1919 1919  URt.Y......2....
00000010: b100 0000 ffff ffff 0000 0000 0000 0200  ................
00000020: 044b 0003                                .K..



{'grill_temp': 116,
 'probe1_temp': 601,
 'grill_target': 225,
 'probe2_temp': 177,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 0,
 'grill_state': 2,
 'grill_mode': 0,
 'fire_state': 4,
 'fire_state_pct': 75}

In [63]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:48:00.281183

00000000: 5552 8000 5902 e100 050a 1432 1919 1919  UR..Y......2....
00000010: c400 0000 ffff ffff 0000 0000 0000 0200  ................
00000020: 044b 0003                                .K..



{'grill_temp': 128,
 'probe1_temp': 601,
 'grill_target': 225,
 'probe2_temp': 196,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 0,
 'grill_state': 2,
 'grill_mode': 0,
 'fire_state': 4,
 'fire_state_pct': 75}

In [67]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T15:51:00.335510

00000000: 5552 7100 5902 e100 050a 1432 1919 1919  URq.Y......2....
00000010: 9600 0000 ffff ffff 0000 0000 0000 0200  ................
00000020: 0400 0003                                ....



{'grill_temp': 113,
 'probe1_temp': 601,
 'grill_target': 225,
 'probe2_temp': 150,
 'probe2_target': 0,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 0,
 'grill_state': 2,
 'grill_mode': 0,
 'fire_state': 4,
 'fire_state_pct': 0}

Still FAn...  ok, finally oFF

In [68]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T16:01:10.515943

00000000: 5552 4000 5902 9600 050a 1432 1919 1919  UR@.Y......2....
00000010: 5200 0000 ffff ffff 0000 0000 0000 0000  R...............
00000020: 0100 0003                                ....



{'grill_temp': 64,
 'probe1_temp': 601,
 'grill_target': 150,
 'probe2_temp': 82,
 'probe2_target': 0,
 'curve_remain_time': -1,
 'warn_code': 0,
 'probe1_target': 0,
 'grill_state': 0,
 'grill_mode': 0,
 'fire_state': 1,
 'fire_state_pct': 0}

Climate stuff

In [73]:
climate_cold =    "0000   55 43 05 0a 14 32 19 19 19 19 21"
climate_average = "0000   55 43 05 06 14 32 19 19 19 19 21"
#                         U  C

In [77]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T16:36:29.587676

00000000: 5552 8e00 5902 9700 0506 1432 1919 1919  UR..Y......2....
00000010: be00 de00 ffff ffff 0000 0000 6f00 0100  ............o...
00000020: 024b 0003                                .K..



{'grill_temp': 142,
 'probe1_temp': 601,
 'grill_target': 151,
 '_08': 5,
 '_09': 6,
 '_10': 20,
 '_11': 50,
 '_12': 25,
 '_13': 25,
 '_14': 25,
 '_15': 25,
 'probe2_temp': 190,
 'probe2_target': 222,
 'curve_remain_time': -1,
 '_21': -1,
 '_22': -1,
 '_23': -1,
 'warn_code': 0,
 '_25': 0,
 '_26': 0,
 '_27': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 2,
 'fire_state_pct': 75,
 '_34': 0,
 '_35': 3}

Low pellet alarm!

In [80]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T16:52:37.645295

00000000: 5552 dd00 5902 e100 0506 1432 1919 1919  UR..Y......2....
00000010: fa00 0000 ffff ffff 8000 0000 6f00 0100  ............o...
00000020: 0364 0003                                .d..



{'grill_temp': 221,
 'probe1_temp': 601,
 'grill_target': 225,
 '_08': 5,
 '_09': 6,
 '_10': 20,
 '_11': 50,
 '_12': 25,
 '_13': 25,
 '_14': 25,
 '_15': 25,
 'probe2_temp': 250,
 'probe2_target': 0,
 'curve_remain_time': -1,
 '_21': -1,
 '_22': -1,
 '_23': -1,
 'warn_code': -128,
 '_25': 0,
 '_26': 0,
 '_27': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 3,
 'fire_state_pct': 100,
 '_34': 0,
 '_35': 3}

added pellets

In [81]:
print(datetime.datetime.utcnow().isoformat())
print()
data = send_command(INFO, mac_to_ip[gmg_mac])
xxd(data)
specunpack(spec, data)

2022-02-14T16:54:21.876194

00000000: 5552 e700 5902 e100 0506 1432 1919 1919  UR..Y......2....
00000010: 4100 0000 ffff ffff 0000 0000 6f00 0100  A...........o...
00000020: 0364 0003                                .d..



{'grill_temp': 231,
 'probe1_temp': 601,
 'grill_target': 225,
 '_08': 5,
 '_09': 6,
 '_10': 20,
 '_11': 50,
 '_12': 25,
 '_13': 25,
 '_14': 25,
 '_15': 25,
 'probe2_temp': 65,
 'probe2_target': 0,
 'curve_remain_time': -1,
 '_21': -1,
 '_22': -1,
 '_23': -1,
 'warn_code': 0,
 '_25': 0,
 '_26': 0,
 '_27': 0,
 'probe1_target': 111,
 'grill_state': 1,
 'grill_mode': 0,
 'fire_state': 3,
 'fire_state_pct': 100,
 '_34': 0,
 '_35': 3}